# Transformers

Autoregressive large language models are build on top of transformers architecture.

Transformer is a neural network architecture based on self-attention mechanisms, avoiding thus recurrence 
connections. This way sequence problems can be solved in parallel, making training large models faster.

Self attention mechanisms relies on compute similarity between embeddings.

To understand in detail the attention mechanisms The Attention is All You Need paper [2] is a good place to
understand the transformer architecture.

I create my personal finds and placed on this repositoy https://github.com/ramonlins/obsidian/tree/master/Papers/transformers/Ashish%20Vaswani.

To access the material is necessary to install obisidian and excalidraw plugin.

In attempt to create a simple example to implement transforms from scratch I created a simple 
overview of the architecture abstracting some parts such as residual connections, stack of layers and 
multiple heads.

![transformer](./images/transformer.png "Figure 1")

The idea here is to create a squence model to predict the next character vowel based on a sequence of three
characters extracted from the portugues alphabet. 


In this simpler ilustration the encoder is composed by a attention layer connected with a feed forward network; 
Its outputs are connected with another attention layer togeter with the output of the masked attention 
layer; This attention layer output is connected to another feedforward layer that pass through a linear and
softmax operation.







## Finding next vowel
pt-alphabet is defined by:

"abcdefghijklmnopqrstuvwxyz"

Dataset

In [605]:
x = "-abcdefghijklmnopqrstuvwxyz"
emb = {}
pos = {}
for i, c in enumerate(x):
    pos[c] = str(i)               # position i
    emb[c] = format(i,'05b') # binary encoding

In [606]:
pos

{'-': '0',
 'a': '1',
 'b': '2',
 'c': '3',
 'd': '4',
 'e': '5',
 'f': '6',
 'g': '7',
 'h': '8',
 'i': '9',
 'j': '10',
 'k': '11',
 'l': '12',
 'm': '13',
 'n': '14',
 'o': '15',
 'p': '16',
 'q': '17',
 'r': '18',
 's': '19',
 't': '20',
 'u': '21',
 'v': '22',
 'w': '23',
 'x': '24',
 'y': '25',
 'z': '26'}

In [607]:
emb

{'-': '00000',
 'a': '00001',
 'b': '00010',
 'c': '00011',
 'd': '00100',
 'e': '00101',
 'f': '00110',
 'g': '00111',
 'h': '01000',
 'i': '01001',
 'j': '01010',
 'k': '01011',
 'l': '01100',
 'm': '01101',
 'n': '01110',
 'o': '01111',
 'p': '10000',
 'q': '10001',
 'r': '10010',
 's': '10011',
 't': '10100',
 'u': '10101',
 'v': '10110',
 'w': '10111',
 'x': '11000',
 'y': '11001',
 'z': '11010'}

In [608]:
# Create all possible combinations of input and ouputs
combinations = "---abcdefghijklmnopqrstuvwxyz"
vowels = 'aeoiu'

vowel_index = {}
for i, v in enumerate(combinations):
    if v in vowels:
        vowel_index[v] = i
        
samples = []
dim_alphapet = len(combinations)
for i in range(dim_alphapet-2):
    input_seq = combinations[i:i+3]
    for vowel, idx in vowel_index.items():
        if idx > i+2:
            next_vowel = vowel
            break
        
        if i+2 >= 23:
            next_vowel = '#'
            break
        
    samples.append([input_seq, next_vowel])
    

In [609]:
class Encoder:
    def __init__(self, x_dim, dm, h):
        #h = h
        #dm = dm
        #dk = dv = dm/h
        #de = x_dim + 1
        
        #Q = torch.nn.Linear(de, dk)
        #K = torch.nn.Linear(de, dk)
        #V = torch.nn.Linear(de, dv)
        pass
    
    def embedding(self, c):
        # get char encode
        ec = emb[c] + pos[c]

        return ec
    
    def attention(self):
        self.matmul()
        self.scale()
        self.softmax()
        self.matmul()
        
        attention = 0
        
        return attention
              
    def matmul(self):
        print('matmul')
    
    def scale(self):
        print('scale')
        
    def softmax(self):
        print('softmax')
    
    
class Decoder(Encoder):
    def __init__(self):
        pass
    
    def embedding(self, _y):
        ed = emb[_y] + pos[_y] 
        led = [int(c) for c in ed]
        emb_dec_t = torch.tensor(led).float().requires_grad_(True)
        ed_pos = int(pos[_y])
        
        return emb_dec_t, ed_pos
    
    def mask_attention(self, 
                       attention, 
                       emb,
                       ec_pos,
                       ed_pos):
        self.matmul()
        self.scale()
        self.mask()
        
        # mask
        if ed_pos <= max(ec_pos):
            print("Position of emb dec is less than all idx in seq, masking..")
            mask_tensor = torch.zeros_like(emb)
            mask_tensor[:] = float('inf')
            
            o = -masked_tensor
        else:
            print("Position of emb dec is bigger than all idx in seq")
            
            o = emb
        self.softmax()
        self.matmul()
        
        return o
   
    def mask(self):
        print("mask")

Build

In [613]:
import random

class transformer:
    def __init__(self):
        self._y = 'a'
        self.encoder = Encoder(x_dim=3, dm=8, h=1)
        self.decoder = Decoder()
        
    def forward(self, x):
        ec = ''
        ec_pos = []
        # encoding embedding
        for c in x:
            ec += self.encoder.embedding(c)
            ec_pos.append(int(pos[c]))
                        
        # encoder
        print("Encoder")
        attention = self.encoder.attention()
        print(f"Encoder attention {attention} \n")
        
        ## Decoder
        print("Decoder")
        emb_dec_t, ed_pos = self.decoder.embedding(self._y)
        
        y = self.decoder.mask_attention(attention, 
                                        emb_dec_t,
                                        ec_pos,
                                        ed_pos)
        
        print(f"Decoder mask output {y}")
        
        self.decoder.attention()
        
        self._y = y
          
        #return y
        return y

In [614]:
transformer = transformer()

In [615]:
x = 'abc'
y = transformer.forward(x)
y

Encoder
matmul
scale
softmax
matmul
Encoder attention 0 

Decoder
matmul
scale
mask
Position of emb dec is less than all idx in seq, masking..
softmax
matmul
Decoder mask output tensor([-inf, -inf, -inf, -inf, -inf, -inf], grad_fn=<NegBackward0>)
matmul
scale
softmax
matmul


tensor([-inf, -inf, -inf, -inf, -inf, -inf], grad_fn=<NegBackward0>)